In [1]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn import model_selection

from sklearn.tree import DecisionTreeClassifier

RANDOM = 51

Import dos dados


In [19]:
# Conjunto de treino
X_train = pd.read_csv('../data/processed/X_train.csv')
Y_train = pd.read_csv('../data/processed/Y_train.csv')

# Conjunto de validação
X_val = pd.read_csv('../data/processed/X_val.csv')
Y_val = pd.read_csv('../data/processed/Y_val.csv')

# Conjunto de teste
X_test = pd.read_csv('../data/processed/X_test.csv')
Y_test = pd.read_csv('../data/processed/Y_test.csv')

In [18]:
X_train

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer
0,0.894297,1,0.936170,0.065570,0.517108,0.003974,0.387882,0.685526
1,0.876510,3,0.978723,0.028582,0.435138,0.000850,0.356810,0.708772
2,0.474404,2,0.723404,0.036328,0.412675,0.002482,0.344122,0.377632
3,0.460820,2,0.638298,0.033025,0.471288,0.002264,0.449249,0.301754
4,0.867086,1,0.042553,0.027981,0.117376,0.001899,0.219834,0.524123
...,...,...,...,...,...,...,...,...
19516,0.894651,2,0.489362,0.084064,0.680006,0.005507,0.665458,0.494737
19517,0.456484,1,0.297872,0.021166,0.215561,0.001417,0.162610,0.753947
19518,0.443078,4,0.936170,0.048006,0.485867,0.003211,0.394873,0.604825
19519,0.877041,1,0.063830,0.028792,0.151592,0.001919,0.241585,0.471930


# Busca de Hiperparâmetros

- Randomized Grid Search
  - 5 K-fold
  - Devemos escolher o modelo conforme: 
    - recall ou F1-score
  
Utilizar n_jobs=-1

In [21]:
def search_params(model, params: dict):

    #numero de folds atraves do metodo k-fold
    kfold = model_selection.StratifiedKFold(n_splits=5,random_state=RANDOM, shuffle=True) # Verificar se fazer o shuffle faz sentido, acho que nao

    search_model = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=5,
        cv=3,
        random_state=51,
        n_jobs=-1
    )

    #treinamento do Random Forest em dez folds
    results = model_selection.cross_val_score(search_model, X_train, Y_train, cv=kfold)
    
    model_rf = search_model.fit(X_train,Y_train)

    print("Random Forest folds:", results, "\nMedia treinamento: " ,results.mean())
    print("Treinamento: ", model_rf.score(X_train,Y_train))
    print("Teste: ", model_rf.score(X_test,Y_test))
    





In [22]:
n_runs = 12  # você pode alterar conforme necessário
best_scores_dt = []
best_params_dt = []

param_distributions_dt = {
    'max_depth': [1, 2, 5, 10, 50, 100],
    'min_samples_split': [2, 3, 5, 7, 10, 20, 50, 100], # minimum number of samples required to split an internal node:
    'min_samples_leaf': [1, 2, 3, 5, 7, 20, 50, 100]    #  minimum number of samples required to be at a leaf node
}


search_params(DecisionTreeClassifier(random_state=42), param_distributions_dt)

Random Forest folds: [0.85352113 0.86116803 0.84964139 0.84349385 0.84938525] 
Media treinamento:  0.8514419302701455
Treinamento:  0.9833000358588188
Teste:  0.8546845124282982
